In [1]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import cdist

# Parameters
SPEED_THRESHOLD = 4  # m/s
MIN_STOP_DURATION = 1.0  # seconds
DIST_THRESH = 5.5
EXPORT_INTERVAL = 0.16  # seconds
PED_BOX = (0.3, 0.3)
BUS_BOX = (10.10,2.43)         

# ----------------------------------
# Helpers
# ----------------------------------

def filter_stopped_vehicles(df, speed_col='speed', time_col='TimeStamp'):
    """Filter out stopped vehicles longer than threshold"""
    df = df.copy()
    df['time_diff'] = df.groupby('Track ID')[time_col].diff()
    df['is_stopped'] = df[speed_col] < SPEED_THRESHOLD
    stop_groups = (df['is_stopped'] != df['is_stopped'].shift()).cumsum()
    df['stop_duration'] = df.groupby(['Track ID', stop_groups])['time_diff'].cumsum()
    return df[~((df['is_stopped']) & (df['stop_duration'] >= MIN_STOP_DURATION))].drop(columns=['time_diff', 'is_stopped', 'stop_duration'])

def get_rotated_corners(x, y, heading, length, width):
    """Return coordinates of rotated bounding box corners"""
    half_l, half_w = length / 2, width / 2
    corners = np.array([
        [ half_l,  half_w],
        [ half_l, -half_w],
        [-half_l, -half_w],
        [-half_l,  half_w]
    ])
    rad = np.radians(heading)
    rot = np.array([
        [np.cos(rad), -np.sin(rad)],
        [np.sin(rad),  np.cos(rad)]
    ])
    return (corners @ rot.T) + np.array([x, y])

def get_closest_corners(row):
    """Get direction vector and min distance between closest corners"""
    ped_corners = get_rotated_corners(row['x_smooth_ped'], row['y_smooth_ped'], row['HA_ped'], *PED_BOX)
    bus_corners = get_rotated_corners(row['x_smooth_bus'], row['y_smooth_bus'], row['HA_bus'], *BUS_BOX)
    dists = cdist(ped_corners, bus_corners)
    idx = np.unravel_index(np.argmin(dists), dists.shape)
    min_dist = dists[idx]
    direction_vec = bus_corners[idx[1]] - ped_corners[idx[0]]
    return min_dist, direction_vec

def calculate_attc(row):
    """Compute ATTC using projection of relative motion on direction vector"""
    min_dist, direction_vec = get_closest_corners(row)
    norm = np.linalg.norm(direction_vec)
    if norm == 0:
        return np.inf
    unit_vec = direction_vec / norm

    rel_v = np.array([
        row['vx_smooth_ped'] - row['vx_smooth_bus'],
        row['vy_smooth_ped'] - row['vy_smooth_bus']
    ])
    rel_a = 0.5 * np.array([
        row['ax_ped'] - row['ax_bus'],
        row['ay_ped'] - row['ay_bus']
    ])

    closing_rate = -np.dot(rel_v, unit_vec) + np.dot(rel_a, unit_vec)
    return min_dist / closing_rate if closing_rate > 0 else np.inf

# ----------------------------------
# Load Data
# ----------------------------------

df_ped = pd.read_csv(r"D:\T\test_codeEVT\nd\ped_smooth.csv")
df_bus = pd.read_csv(r"D:\T\test_codeEVT\nd\bus_smooth.csv")
df_bus['speed'] = np.sqrt(df_bus['vx_smooth']**2 + df_bus['vy_smooth']**2)

# Filter stopped vehicles
print(f"Original busrcycle count: {len(df_bus)}")
df_bus = filter_stopped_vehicles(df_bus, speed_col='speed')
print(f"Filtered busrcycle count: {len(df_bus)}")
print(df_bus['speed'].mean())
# Calculate yaw rates
for df in [df_ped, df_bus]:
    df['yaw_rate'] = df.groupby('Track ID').apply(
        lambda x: x['HA'].diff() / x['TimeStamp'].diff()
    ).reset_index(level=0, drop=True)

# Round timestamps
df_ped['Time_rounded'] = (df_ped['TimeStamp'] / EXPORT_INTERVAL).round() * EXPORT_INTERVAL
df_bus['Time_rounded'] = (df_bus['TimeStamp'] / EXPORT_INTERVAL).round() * EXPORT_INTERVAL


Original busrcycle count: 14925
Filtered busrcycle count: 2281
4.436940186027145


C:\Users\IST LAB 1\AppData\Local\Temp\ipykernel_19624\859294420.py:87: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df['yaw_rate'] = df.groupby('Track ID').apply(
C:\Users\IST LAB 1\AppData\Local\Temp\ipykernel_19624\859294420.py:87: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df['yaw_rate'] = df.groupby('Track ID').apply(


In [3]:

# ----------------------------------
# Merge and Process
# ----------------------------------

merged = pd.merge(
    df_ped, 
    df_bus, 
    on='Time_rounded', 
    suffixes=('_ped', '_bus')
)

merged['Center_dist'] = np.hypot(
    merged['x_smooth_ped'] - merged['x_smooth_bus'],
    merged['y_smooth_ped'] - merged['y_smooth_bus']
)

# Filter by center threshold
results = merged[merged['Center_dist'] <= DIST_THRESH].copy()

# Calculate ATTC using corrected method
results['ATTC'] = results.apply(calculate_attc, axis=1)
results[['mindis', 'direction_vec']] = results.apply(
    lambda row: pd.Series(get_closest_corners(row)),
    axis=1
)
# Output
output = results[['Track ID_ped', 'Track ID_bus', 'TimeStamp_ped', 'ATTC']].rename(columns={
    'Track ID_ped': 'Ped_ID',
    'Track ID_bus': 'bus_ID',
    'TimeStamp_ped': 'TimeStamp'
})
print(output)
len(results[(results['ATTC']<1) & (results['mindis']<1)][['Track ID_ped', 'Track ID_bus']].drop_duplicates())

      Ped_ID  bus_ID  TimeStamp      ATTC
65       415     439     166.96  0.429555
66       415     439     167.12  0.444046
67       415     439     167.28  0.488769
68       415     439     167.44  0.486187
69       415     439     167.60       inf
...      ...     ...        ...       ...
8837   15503   15474    5287.32       inf
8838   15503   15474    5287.48       inf
8839   15503   15474    5287.64       inf
8840   15503   15474    5287.80       inf
8841   15503   15474    5287.96       inf

[214 rows x 4 columns]


4

In [4]:
results[(results['ATTC']<1) & (results['mindis']<1)][['Track ID_ped', 'Track ID_bus']].drop_duplicates()

,Track ID_ped,Track ID_bus
3207,5556,5610
5059,9253,9326
5558,10279,10291
5601,10292,10291


In [5]:
results[(results['ATTC']<1) & (results['mindis']<1)][['Track ID_ped', 'Track ID_bus','TimeStamp_ped']]

,Track ID_ped,Track ID_bus,TimeStamp_ped
3207,5556,5610,2037.08
5059,9253,9326,3366.60
5060,9253,9326,3366.76
5558,10279,10291,3670.64
5572,10279,10291,3672.88
5573,10279,10291,3673.04
5601,10292,10291,3673.04


In [6]:
# Create new DataFrame with selected and renamed columns
conflict_summary = results[[
    'Track ID_ped',        # Pedestrian ID
    'Track ID_bus',       # Vehicle ID
    'Type_bus',           # Vehicle type
    'TimeStamp_ped',       # Timestamp
    'mindis',              # Minimum corner distance
    'ATTC'                 # ATTC value
]].copy()

# Rename columns
conflict_summary.rename(columns={
    'Track ID_ped': 'PED_ID',
    'Track ID_bus': 'VEH_ID',
    'Type_bus': 'TYPE',
    'TimeStamp_ped': 'TIMESTAMP',
    'mindis': 'MIN_COR_DIS',
    'ATTC': 'ATTC'
}, inplace=True)

# Preview
print(conflict_summary.head())


    PED_ID  VEH_ID  TYPE  TIMESTAMP  MIN_COR_DIS      ATTC
65     415     439   Bus     166.96     3.086744  0.429555
66     415     439   Bus     167.12     3.106553  0.444046
67     415     439   Bus     167.28     4.736058  0.488769
68     415     439   Bus     167.44     4.666619  0.486187
69     415     439   Bus     167.60     4.993631       inf


In [7]:
conflict_summary.to_csv(r"D:\T\test_codeEVT\ATTC_Data/Bus_Ped.csv", index=False)